In [1]:
# define rooms and items
count = 50
from colorama import Fore, Back, Style

from PIL import Image 
# ============================ Games ==================================
game1 = {
    "name": "game_1",
    "type": "game",
    "question": " How much is 1 + 1? and it is not 7 BTW. If you dont get it is beacause you are not Spanish and you are too old or maybe I am",
    "value": 5,
    "btw": "I think you get it. It wasn´t so difficult right? '(It's a rhetorical question, you don't have to answer me. Well...you can't)'" + '\n'
}

game2 = {
    "name": "game_2",
    "type": "game",
    "question": " Can God create a rock he cannot lift? I will help you: the possible answers are: 'yes', 'no' or 'maybe'. If you want to discuss it with the game answer try next time. In this instance I'm like God, what I decide is the truth.",
    "answer": "maybe",
    "value": 5 ,
    "btw": "yes it is maybe, because we need to get more information. If god is powerfull it mean he can create. However if he cant not lift it he is not powerful enough" + '\n'
}

game3 = {
    "name": "game_3",
    "type": "game",
    "question": "it is a pikachu, do you have a pokeball? i dont think so. Bad luck this happen only one time in life. so... I will give you the anwser this time type 'no'",
    "answer": "no",
    "value": 5,
    "btw": "Sorry, a pikachu is too much for you try maybe with a magikarp" + '\n',   
}

game4 = {
    "name": "game_4",
    "type": "game",
    "question": "Which team is going to win?",
    "answer": "spooky pumpkins", 
    "value": 5 ,
    "btw": "Yes, you're right! The Spooky Pumpkins are going to win. If it doesn't happen this invisible rabbit will disappear!. so you know who to vote for" + '\n',   
}

game5 = {
    "name": "game_5",
    "type": "game",
    "question": "If a rooster lays an egg on a roof, which way does it fall to? Some options are: 'left', 'right', 'both' or 'none'",
    "answer": "none", 
    "value": 5,
    "btw": "If you said left or right you were wrong, at least in our world. If it was a snail, you would've nailed it!" + '\n',   
}

game6 = {
    "name": "game_6",
    "type": "game",
    "question": "Is darkness black?",
    "answer": "no", 
    "value": 5,
    "btw": "In science, black is the absence of light and color is a phenomenon of light. However, a black object or black images printed on white paper are made up of pigments, not light. So artists should use the darkest paint color to approximate black." + '\n'   
}

bonus_door = {
    "name": "bonus_door",
    "type": "game",
    "question": "This is a bonus room, you dont need to use a Key you need to use a the code. It should be something like the number pi. Good Luck! ",
    "answer": "3,14",    
}

# ============================ Doors ==================================


door_a = {
    "name": "door a",
    "type": "door",
}

door_b = {
    "name": "door b",
    "type": "door",
}

door_c = {
    "name": "door c",
    "type": "door",
}

door_d = {
    "name": "door d",
    "type": "door",
}

door_e = {
    "name": "door e",
    "type": "door",
}

door_f = {
    "name": "door f",
    "type": "door",
}

door_g = {
    "name": "door g",
    "type": "door",
}

door_h = {
    "name": "door h",
    "type": "door",
}

door_i = {
    "name": "door i",
    "type": "door",
}

door_j = {
    "name": "door j",
    "type": "door",
}

door_k = {
    "name": "door k",
    "type": "door",
}

door_l = {
    "name": "door l",
    "type": "door",
}

door_m = {
    "name": "door m",
    "type": "door",
}



# ============================ Keys ==================================

key_a = {
    "name": "key for door a",
    "type": "key",
    "target": door_a,
    "value": 8,
}

key_b = {
    "name": "key for door b",
    "type": "key",
    "target": door_b,
}

key_c = {
    "name": "key for door c",
    "type": "key",
    "target": door_c,
}

key_d = {
    "name": "key for door d",
    "type": "key",
    "target": door_d,
}

key_e = {
    "name": "key for door e",
    "type": "key",
    "target": door_e,
}

key_f = {
    "name": "key for door f",
    "type": "key",
    "target": door_f,
}

key_g = {
    "name": "key for door g",
    "type": "key",
    "target": door_g,
}

key_h = {
    "name": "key for door h",
    "type": "key",
    "target": door_h,
}

key_i = {
    "name": "key for door i",
    "type": "key",
    "target": door_i,
}

key_j = {
    "name": "key for door j",
    "type": "key",
    "target": door_j,
}


key_k = {
    "name": "key for door k",
    "type": "key",
    "target": door_k,
}


key_l = {
    "name": "key for door l",
    "type": "key",
    "target": door_l,
}

key_m = {
    "name": "key for door m",
    "type": "key",
    "target": door_m,    
}

# ============================ Furnitures ==================================

# ========= Game Room =========

piano = {
    "name": "piano",
    "type": "furniture",
    "value": 7,
}

couch = {
    "name": "couch",
    "type": "furniture",
}

round_table = {
    "name": "round table",
    "type": "furniture"
}
# ========= Bathroom =========

bathtub = {
    "name": "bathtub",
    "type": "furniture",
}

sink = {
    "name": "sink",
    "type": "furniture",
}

toilet = {
    "name": "toilet",
    "type": "furniture",
    "value": 2,
}

# ========= Bedroom 1 =========

queenbed = {
    "name": "queen bed",
    "type": "furniture",
}

dressing_table = {
    "name": "dressing table",
    "type": "furniture",   
}

# ========= Bedroom 2 =========

doublebed = {
    "name": "double bed",
    "type": "furniture",
}

dresser = {
    "name": "dresser",
    "type": "furniture",
}

desktop = {
    "name": "desktop",
    "type": "furniture",    
}

desktop_chair = {
    "name": "desktop chair",
    "type": "furniture",
}

# ========= Living Room =========

living_room_table = {
    "name": "living room table",
    "type": "furniture",
}



# ========= Dungeon =========

bucket = {
    "name": "bucket",
    "type": "furniture",
}

torture_table = {
    "name": "torture table",
    "type": "furniture",
}

blood_table = {
    "name": "blood table",
    "type": "furniture",
}

# ========= corridor =========


# ========= Lab =========

flasks_cabinet = {
    "name": "flasks cabinet",
    "type": "furniture",
}

table_lab = {
    "name": "randowm table",
    "type": "furniture",
}

# ========= Kitchen =========

kitchen_table = {
    "name": "kitchen table",
    "type": "furniture",
}

kitchen_sink = {
    "name": "kitchen sink",
    "type": "furniture",
}

kitchen_countertop = {
    "name": "kitchen countertop",
    "type": "furniture",
}

stove = {
    "name": "stove",
    "type": "furniture",
}

# ========= Bait Room =========

toys_trunk = {
    "name": "toys trunk",
    "type": "furniture",
}

candy_cabinet = {
    "name": "candy cabinet",
    "type": "furniture",
}

console_station = {
    "name": "Xbox",
    "type": "furniture",
}

# ========= Dining Room =========

cabinet = {
    "name": "cabinet",
    "type": "furniture",
}

dining_table = {
    "name": "dining table",
    "type": "furniture",
}


# ========= Entrance =========

bench = {
    "name": "bench",
    "type": "furniture",
}

# ============================ Rooms ==================================

game_room = {
    "name": "the Game Room",
    "type": "room",
    "map": "Map1.png",
}

bedroom_1 = {
    "name": "the Bedroom",
    "type": "room",
    "map": "Map3.png",
}

bathroom = {
    "name": "the Bathroom",
    "type": "room",
    "map": "Map2.png",
}

bedroom_2 = {
    "name": "the Small bedroom",
    "type": "room",
    "map": "Map4.png",
}

living_room = {
    "name": "the Living Room",
    "type": "room",
    "map": "Map6.png",
}

dungeon = {
    "name": "the Dungeon",
    "type": "room",
    "map": "Map5.png",
}

kitchen = {
    "name": "the Kitchen",
    "type": "room",
    "map": "Map9.png",
}

lab = {
    "name": "the Lab",
    "type": "room",
    "map": "Map8.png",
}

bait_room = {
    "name": "the Bait Room",
    "type": "room",
    "map": "Map11.png",
}

dining_room = {
    "name": "the Dining Room",
    "type": "room",
    "map": "Map10.png",
}

corridor = {
    "name": "the Corridor",
    "type": "room",
    "map": "Map7.png",
}

entrance = {
    "name": "the Entrance",
    "type": "room",
    "map": "Map12.png",
}

outside = {
    "name": "Outside",
    "map": "CompleteMap.png"
}

all_rooms = [game_room, bathroom, bedroom_1, bedroom_2, living_room, dungeon, corridor, kitchen, lab, bait_room, dining_room, entrance, outside]

all_doors = [door_a, door_b, door_c, door_d, bonus_door, door_e, door_f, door_g, door_h, door_i, door_j, door_k, door_l]

# define which items/rooms are related

object_relations = {
    # ===== Rooms =====
    
    "the Game Room": [couch, piano, round_table, door_a, door_b, bonus_door, game1],
    "the Living Room": [living_room_table, door_e, door_f],
    "the Small bedroom": [doublebed, dresser, desktop, door_c, game2],
    "the Bedroom": [queenbed, dressing_table, door_b, door_c, door_d, door_e, game3],
    "the Bathroom": [toilet, door_a, bathtub, sink],
    "the Dungeon": [door_d, torture_table, blood_table, bucket, game4],
    "the Kitchen": [door_h, door_i, kitchen_table, kitchen_countertop, stove, game5],
    "the Lab": [door_g, door_h, door_j, flasks_cabinet, table_lab, game6],
    "the Dining Room": [dining_table, door_i, cabinet],
    "the Bait Room": [door_k, door_j, candy_cabinet, console_station, toys_trunk],
    "the Entrance": [bench, door_l, door_k],
    "the Corridor": [door_f, door_g],
    
    # ===== keys =====
    
    "piano": [key_a],
    "queen bed": [key_c],
    "dresser": [key_d],
    #"doublebed": [key_f],
    "toilet": [key_b],
    "bucket": [key_e],
    "desktop": [key_g],
    "living room table": [key_f],
    "flasks cabinet": [key_h],
    "stove": [key_i],
    "candy cabinet": [key_k],
    "cabinet": [key_j],
    "bench": [key_l],
    
    
    # ===== Doors =====

    "door a": [game_room, bathroom],
    "door b": [bedroom_1, game_room],
    "door c": [bedroom_1, bedroom_2],
    "door d": [bedroom_1, dungeon],
    "door e": [bedroom_1, living_room],
    "door f": [living_room, corridor],
    "door g": [lab, corridor],
    "door h": [kitchen, lab],
    "door i": [dining_room, kitchen],
    "door j": [lab, bait_room],
    "door k": [entrance, bait_room],
    "door l": [entrance, outside],
    "bonus_door": [game_room],
    "outside": [door_l],
}  


# define game state. Do not directly change this dict. 
# Instead, when a new game starts, make a copy of this
# dict and use the copy to store gameplay state. This 
# way you can replay the game multiple times.

INIT_GAME_STATE = {
    "current_room": game_room,
    "keys_collected": [],
    "target_room": outside
}

In [2]:
def linebreak():
    """
    Print a line break
    """
    print("\n\n")

def start_game():
    """
    Start the game
    """
    print("You wake up on a couch and find yourself in a strange house with no windows which you have never been to before. You don't remember why you are here and what had happened before. You feel some unknown danger is approaching and you must get out of the house, NOW!" + '\n')
    print("You start with 50 points and the interaction with each object in each room will take points off. Sometimes. However you can get extra points if you play each room's minigame. So good luck, you will need it." + "\n" + "BTW if you find a problem, it's not the game's fault, it's yours for not understanding how to play muahaha.")
    play_room(game_state["current_room"])

    
def start_game_because_you_die():
    """
    Start the game
    """
    print(Fore.RED + "You died, start again")
    play_room(game_state["current_room"])


def play_room(room):
    """
    Play a room. First check if the room being played is the target room.
    If it is, the game will end with success. Otherwise, let player either 
    explore (list all items in this room) or examine an item found here.
    """
    game_state["current_room"] = room
    if(game_state["current_room"] == game_state["target_room"]):
        print(Fore.RED + "Congrats! You escaped the room!")
    else:
        
        print(Fore.BLUE + "You are now in " + room["name"]+", "+ meet_monster()+" welcomes you!" + '\n')
        intended_action = input("What would you like to do? Type '0' to explore the room, '1' to examine an item, '2' to check the map or '3' to see you current points" + '\n').strip()
        if intended_action == "0":
            explore_room(room)
            play_room(room)
        elif intended_action == "1":
            examine_item(input("What would you like to examine?"+'\n').strip())
        elif intended_action == "2":
            map = Image.open(room["map"])
            map.show()
            play_room(room)
        elif intended_action == "3":
            print("You have ",count," points")
            play_sound('pain.mp3')
            play_room(room)
        else:
            print(Fore.RED + "Not sure what you mean. Type '0' to explore the room, '1' to examine an item, '2' to check the map or '3' to see you current points'."+'\n')
            play_room(room)
        linebreak()

def explore_room(room):
    play_sound('dark_harp.mp3')
    """
    Explore a room. List all items belonging to this room.
    """
    items = [i["name"] for i in object_relations[room["name"]]]
    print("You explore the room. This is " + room["name"] + ". You find " + ", ".join(items))

def get_next_room_of_door(door, current_room):
    play_sound('ghosts.mp3')
    """
    From object_relations, find the two rooms connected to the given door.
    Return the room that is not the current_room.
    """
    connected_rooms = object_relations[door["name"]]
    for room in connected_rooms:
        if(not current_room == room):
            return room


def examine_item(item_name):
    """
    Examine an item which can be a door or furniture.
    First make sure the intended item belongs to the current room.
    Then check if the item is a door. Tell player if key hasn't been 
    collected yet. Otherwise ask player if they want to go to the next
    room. If the item is not a door, then check if it contains keys.
    Collect the key if found and update the game state. At the end,
    play either the current or the next room depending on the game state
    to keep playing.
    """
    current_room = game_state["current_room"]
    next_room = ""
    output = None
    global  count
    for item in object_relations[current_room["name"]]:
        if(item["name"] == item_name):
            output = "You examine " + item_name + ". "
            if(item["type"] == "door"):
                have_key = False
                for key in game_state["keys_collected"]:
                    if(key["target"] == item):
                        have_key = True
                if(have_key):
                    output += "You unlock it with a key you have."
                    next_room = get_next_room_of_door(item, current_room)
                else:
                    output += "It is locked but you don't have the key." + '\n'
            elif(item["type"] == "game"):
                print("You fine a mini game called: " + item["name"] + "and you need to answer it" + item["question"])
                play_game = input(item["question"]).strip()
                if play_game != item["answer"]:
                    print(Fore.RED + "Sorry, we can see you are not SMART enough to play it (yes! I want to emphasize it), our recommendation is to try to google it and try again. However, I dont want to waste my time so try to check other things. BYE!")
                else:
                    print(Fore.RED + "You win")
                    count = count + item["value"]
                    if count < 0:
                        print(Back.RED + "You ran out of points!")
                        start_game_because_you_die ()
            else:
                if(item["name"] in object_relations and len(object_relations[item["name"]])>0):
                    item_found = object_relations[item["name"]].pop()
                    game_state["keys_collected"].append(item_found)
                    output += "You find " + (Fore.RED + item_found["name"]) + "."
                else:
                    output += "There isn't anything interesting about it."
            print(output + '\n')
            break

    if(output is None):
        print("The item you requested is not found in the current room.")
    
    if(next_room and input("Do you want to go to the next room? Ener 'yes' or 'no'"+'\n').strip() == 'yes'):
        play_room(next_room)
    else:
        play_room(current_room)

def meet_monster():
  import pandas as pd
  m = pd.read_csv (r'20171019_HalloweenCostumes_Frightgeist2.csv', header=2)
  m1 = m.sample(1).iloc[0,0]
  return m1

def play_sound(sound):
    from playsound import playsound
    return playsound(sound)

In [3]:
game_state = INIT_GAME_STATE.copy()

start_game()

You wake up on a couch and find yourself in a strange house with no windows which you have never been to before. You don't remember why you are here and what had happened before. You feel some unknown danger is approaching and you must get out of the house, NOW!

You start with 50 points and the interaction with each object in each room will take points off. Sometimes. However you can get extra points if you play each room's minigame. So good luck, you will need it.
BTW if you find a problem, it's not the game's fault, it's yours for not understanding how to play muahaha.
You are now in the Game Room, Beyonce welcomes you!

What would you like to do? Type '0' to explore the room, '1' to examine an item, '2' to check the map or '3' to see you current points
1
What would you like to examine?
piano
You examine piano. You find key for door a.

You are now in the Game Room, Beetlejuice welcomes you!

What would you like to do? Type '0' to explore the room, '1' to examine an item, '2' to che

What would you like to examine?
flasks cabinet
You examine flasks cabinet. You find key for door h.

You are now in the Lab, Jedi welcomes you!

What would you like to do? Type '0' to explore the room, '1' to examine an item, '2' to check the map or '3' to see you current points
1
What would you like to examine?
door h
You examine door h. You unlock it with a key you have.

Do you want to go to the next room? Ener 'yes' or 'no'
yes
You are now in the Kitchen, Wilma Flintstone welcomes you!

What would you like to do? Type '0' to explore the room, '1' to examine an item, '2' to check the map or '3' to see you current points
2
You are now in the Kitchen, Diego de la Vega welcomes you!

What would you like to do? Type '0' to explore the room, '1' to examine an item, '2' to check the map or '3' to see you current points
0
You explore the room. This is the Kitchen. You find door h, door i, kitchen table, kitchen countertop, stove, game_5
You are now in the Kitchen, Hipster welcomes you!

Wh

Do you want to go to the next room? Ener 'yes' or 'no'
yes
You are now in the Lab, Robin Hood welcomes you!

What would you like to do? Type '0' to explore the room, '1' to examine an item, '2' to check the map or '3' to see you current points
1
What would you like to examine?
1
The item you requested is not found in the current room.
You are now in the Lab, Shadow welcomes you!

What would you like to do? Type '0' to explore the room, '1' to examine an item, '2' to check the map or '3' to see you current points
1
What would you like to examine?
1
The item you requested is not found in the current room.
You are now in the Lab, Ursula welcomes you!

What would you like to do? Type '0' to explore the room, '1' to examine an item, '2' to check the map or '3' to see you current points
door j
Not sure what you mean. Type '0' to explore the room, '1' to examine an item, '2' to check the map or '3' to see you current points'.

You are now in the Lab, Pink Ladies welcomes you!

What would you 